In [4]:
import cv2
import numpy as np


In [5]:
img = cv2.imread('1.jpg')
img = cv2.resize(img,(640,640))
img_copy = img.copy()
mask = np.zeros(img.shape[:2], np.uint8) # 创建掩码

bgdModel = np.zeros((1,65), np.float64)
fgdModel = np.zeros((1,65), np.float64)

In [6]:
rect = cv2.selectROI(img)  # 选择ROI

x,y,w,h = rect  #

# 绘制矩形
cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0,0,255), 3)

# 保存图像
cv2.imwrite('2.png', img_copy)

cv2.grabCut(img, mask, rect, bgdModel, fgdModel,5,cv2.GC_INIT_WITH_RECT)

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 array([[ 8.40699374e-02,  1.08574983e-01,  3.95492022e-01,
          4.36851468e-02,  3.68177910e-01,  1.72396437e+02,
          2.20992507e+02,  2.25846788e+02,  4.00180995e+01,
          3.22609838e+01,  2.06231244e+02,  2.33657166e+02,
          2.25505395e+02,  2.21263752e+02,  6.93679485e+01,
          5.61790314e+01,  5.73334845e+01,  2.03885028e+02,
          1.94047941e+02,  1.94144565e+02,  1.17487295e+03,
         -1.86351853e+02, -3.84512804e+02, -1.86351853e+02,
          4.20388416e+02,  4.50378747e+02, -3.84512804e+02,
          4.50378747e+02,  5.49469386e+02,  3.31911072e+02,
          2.94114274e+02,  4.14726292e+02,  2.94114274e+02,
          2.70749067e+02,  3.61870645e+02,  4.14726292e+02,
          3.61870645e+02,  6.65760605e+02,  9.91769021

In [7]:
mask2 = np.where((mask==2) | (mask==0), 0,1).astype('uint8')

In [8]:
cv2.imshow("Mask", mask * 80)

cv2.imshow("Mask2", mask2 * 255)

# cv2.imwrite("mask_1.png", mask * 80)
# cv2.imwrite("mask2_1.png", mask * 255)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [9]:
img = img * mask2[:,:,np.newaxis]

img_mask = img.copy()

mask2 = mask2 * 255
mask_copy = mask2.copy()

In [10]:
class Sketcher:
  def __init__(self, windowname, dests, colors_func):
    self.prev_pt = None
    self.windowname = windowname
    self.dests = dests
    self.colors_func = colors_func
    self.dirty = False
    self.show()
    cv2.setMouseCallback(self.windowname, self.on_mouse)

  def show(self):
    cv2.imshow(self.windowname, self.dests[0])
    cv2.imshow(self.windowname + ":mask", self.dests[1])

  # 鼠标处理
  def on_mouse(self, event, x, y, flags, param):
    pt = (x,y)
    if event == cv2.EVENT_LBUTTONDOWN:
      self.prev_pt = pt
    elif event == cv2.EVENT_LBUTTONUP:
      self.prev_pt = None
    if self.prev_pt and flags & cv2.EVENT_FLAG_LBUTTON:
      for dst, color in zip(self.dests, self.colors_func()):
        cv2.line(dst, self.prev_pt, pt, color, 5)
      self.dirty = True
      self.prev_pt = pt
      self.show()

In [11]:
sketch = Sketcher('image', [img_mask, mask2], lambda : ((255,0,0),255))

In [13]:
cv2.imwrite("img_mask_res.png",img_mask)

True

In [12]:
while True:
  ch = cv2.waitKey()
  if ch == 27:
    print("exiting....")
    cv2.imwrite("img_mask_res.png",img_mask)
    cv2.imwrite("mask_res.png",mask2)
    break
  
  elif ch == ord('r'):
    print("reseting...")
    img_mask = img.copy()
    mask2 = mask_copy.copy()
    sketch = Sketcher('image', [img_mask, mask2], lambda : ((255,0,0),255))
    sketch.show()

  elif ch == ord('b'):
    print("drawing background...")
    sketch = Sketcher('image', [img_mask, mask2], lambda : ((0,0,255),0))
    sketch.show()

  elif ch == ord('f'):
    print("drawing foreground...")
    sketch = Sketcher('image', [img_mask, mask2], lambda : ((255,0,0),255))
    sketch.show()

  else:
    print("performing grabcut...")
    mask2 = mask2 // 255
    cv2.grabCut(img, mask2, None, bgdModel, fgdModel,5,cv2.GC_INIT_WITH_MASK)
    mask2 = np.where((mask==2) | (mask==0), 0,1).astype('uint8')
    img_mask = img * mask2[:,:,np.newaxis]
    maks2 = mask2 * 255
    print("switching back to foreground...")
    sketch = Sketcher('image', [img_mask, mask2], lambda : ((255,0,0),255))
    sketch.show()

exiting....
